In [43]:
import mne
import numpy as np
import os.path as op
from tools import files
from scipy.stats import trim_mean, ttest_1samp, sem, ttest_rel, ttest_ind
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
# functions
trim = lambda x: trim_mean(x, 0.1, axis=0)

In [3]:
main_path = op.join("/cubric", "scratch", "c1557187", "act_mis")
meg_path = op.join(main_path, "MEG")

In [4]:
participants = files.get_folders_files(meg_path, wp=False)[0]
participants.sort()
participants.remove("0004")
participants.remove("0009")
participants.remove("0042")
print(participants)

['0001', '0003', '0008', '0014', '0015', '0016', '0017', '0019', '0021', '0027', '0030', '0032', '0034', '0035', '0038', '0039', '0040', '0041', '0043']


In [5]:
results_path = op.join(main_path, "RESULTS", "TD_SENSOR_SPACE")
data_analysis_save = op.join(results_path, "data_analysis.npy")
data_viz_save = op.join(results_path, "data_viz.npy")

In [27]:
odd = []
reg = []
for pp in participants:
    print(pp)
    participant_path = op.join(meg_path, pp, "new_v1")
    participant_files = files.get_files(participant_path, "epochs-TD", "-epo.fif")[2]
    odd_epo = []
    reg_epo = []
    for i in participant_files:
        epochs = mne.read_epochs(i, preload=True)
        epochs = epochs.pick_types(meg=True, ref_meg=False, eog=False)
        info = epochs.info
        times = epochs.times
        epochs = epochs.apply_baseline((-0.1, 0.0))
        epochs = epochs.apply_baseline((1.6, 2.6))
        odd_epo.append(epochs["40"].get_data())
        reg_epo.append(epochs["30"].get_data())
    
    odd_epo = np.vstack(odd_epo)
    odd_epo = trim_mean(odd_epo, 0.1, axis=0)
    odd.append(odd_epo)
    reg_epo = np.vstack(reg_epo)
    reg_epo = trim_mean(reg_epo, 0.1, axis=0)
    reg.append(reg_epo)

0001
0003
0008
0014
0015
0016
0017
0019
0021
0027
0030
0032
0034
0035
0038
0039
0040
0041
0043


In [31]:
odd_X = np.array(odd)
reg_X = np.array(reg)

In [46]:
t = np.zeros((274, 775))
pv = np.zeros((274, 775))

for channel in range(274):
    for time in range(775):
        o = odd_X[:,channel,time]
        r = reg_X[:,channel,time]
        s, p = ttest_ind(o, r)
        t[channel, time] = s
        pv[channel, time] = p
        

In [103]:
p_map = pv < 0.01
t_thr = t * p_map

In [104]:
inv_pv = np.ones(pv.shape) - pv

In [105]:
plt.imshow(p_map,origin="upper", cmap="gray")

In [114]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.imshow(t,origin="upper", cmap="gray")


In [119]:
mne.viz.plot_topomap(t[:,ix], info, res=200, mask=p_map[:,ix], show=False)

(<matplotlib.image.AxesImage at 0x7f964cc49550>,
 <matplotlib.contour.QuadContourSet at 0x7f964cc49320>)

In [93]:
ix/250 - 2

0.33199999999999985